In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
folder='Crime_Boston'
file='crimes_in_boston.csv'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Crime_Boston/crimes_in_boston.csv')# To display the top 5 rows 
df.head(5)

In [ ]:
df.dtypes

In [ ]:
#df = df.drop(['Lat', 'Long'], axis=1)
#df.head(5)

Podemos ver que el Lat y Long tienen 19.998 registros menos que Location. 
Shooting es un true unicamente cuando se registro un tiroteo dentro del crimen, a pesar de que sean pocas, es importante considerarlo.

In [ ]:
df.count()

Eliminar duplicados

In [ ]:
df.shape

In [ ]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

Esto quiere decir que de las 319073 filas hay 23 duplicadas que eliminaremos.

In [ ]:
df = df.drop_duplicates()
df.shape

In [ ]:
print(df.isnull().sum())

1. Los que no tienen Lat y Long, vemos que es porque se seteo una Location de (0.00000000, 0.00000000). Lo podemos identificar dado que los 19998 que no tenian Lat y Long es igual a la frecuencia de (0.00000000, 0.00000000) en el campo Location.

2. Dada la repeticion del Incident number, siendo "I162030584" el mas repetido, 13 veces, podemos concluir que cada incidente puede tener mas de un registro, indicando distintos OFFENSE_CODE/GROUP/DESCRIPTION. Realizado para agregar detalle a ese incidente.

3. B2 es el distrito con mayor cantidad de registros, sin embargo, al repetirse el Incident Number, tendriamos que eliminar los repetidos para saber cual fue el número exacto de incidentes.

In [ ]:
summary = df.describe(include='all').transpose()
summary